In [ ]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/law.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents


# 문서화 실행
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_precedent_documents(load_json(file_paths["precedent"]))
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs
all_docs = precedent_docs

In [89]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings()

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb"
)

# 남은 청크를 순차적으로 추가
for i in range(batch_size, len(all_splits), batch_size):
    batch = all_splits[i:i+batch_size]
    vectorstore.add_documents(
        documents=batch,
        embedding=embedding_model
    )

vectorstore.persist()  


In [90]:
vectorstore.get()

{'ids': ['7e84b59c-7140-47dc-9b7c-f3a4608f5fa0',
  '8d7d7350-5942-46ef-9fad-19f01dc59f77',
  'ffa95a58-4ad7-4107-ae13-c622e5e45a9b',
  '90f413c4-5578-42ed-877b-f996e82f0d6d',
  '2412fde0-8fce-43e4-9b9c-235ecb21f90c',
  '5175ad97-abad-4c02-9c26-907e06f181fe',
  '8e190bb3-2d3c-4cc1-a26d-54b7380e607d',
  '608298ea-7875-4342-a4f7-aa97310c5b4c',
  '7fb8f92d-e4c2-4173-a910-ffc55b4c1b27',
  'd41597a2-a2e9-46e5-b340-1a4fdc71c03c',
  'da83cfe7-a92b-4ac4-8f83-5263dfac3ef2',
  '0e7bc5ac-d0a0-4f49-aab8-6dce98a9bebb',
  'ae989ea3-fab9-4edb-a8f5-1a7591abb02f',
  'dad2a8f7-2ef3-41b5-b000-3af45f00f172',
  'd6166fe2-b795-481a-a9c6-24ca655dbf7e',
  'e4585a21-92f5-46d4-9c0b-515ec231d5e3',
  '3af5c6e7-7033-463b-ba82-85090c107979',
  '430a2912-d577-4f20-a0d0-a2cc58bd53b3',
  'a28b1276-431b-47a6-b2cf-7b143717fbb5',
  '8a8e2071-a76f-45fb-a0d0-431a6783769c',
  '5e4265f0-6317-44e6-ae29-08d5f80ba309',
  '66d740b9-0e72-4560-b69c-45103ac3e6cd',
  '9a788ede-0818-4f1d-8778-0cd4c931f66c',
  'c853bf49-cd61-433d-b13b-

In [91]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
다음 '사고 상황 설명'에 대해 '문서'의 내용만 참고하여 과실 비율 및 법적 판단을 생성해 주세요.

사고 상황: {question}

문서 내용: {context}

답변 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 사고 원인 및 판단 근거:
   - [사고의 전개, 각 차량의 행위, 과실 요소 등을 구체적으로 설명]
3. 관련 법률 조항:
   - [예: 도로교통법 제10조 제2항, 제27조 제1항 등]
4. 참고 판례:
   - [예: 대법원 2023다12345, 서울중앙지법 2022가단123456 등]

조건:
- 반드시 문서 내용만 참고해 판단하세요.
- 법률 조항과 판례가 명시되어 있지 않으면 유사하거나 추정 가능한 근거를 제시해도 됩니다.
- 전체 답변은 포맷에 맞게 간결하고 전문적으로 작성하세요.
""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 4. QA 체인 재구성 (수정 부분)
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 5. 실제 질의 실행
query = "B차량이 비보호좌회전 표지가 없는 교차로에서 녹색등(좌회전 화살표 없음)에 좌회전 시도, 직진하는 A차량과 충돌한 사고가 발생했어. 과실비율이 어떻게 돼??"

res = qa_chain.invoke({"query": query})

# 6. 출력
print("✅ 답변:\n", res["result"])


✅ 답변:
 1. 과실비율: A차량 70% vs B차량 30%

2. 사고 원인 및 판단 근거:
   - 사고는 비보호좌회전 표지가 없는 교차로에서 발생했습니다. B차량은 녹색등에서 좌회전을 시도했으나, 좌회전 화살표가 없는 상황에서 A차량과 충돌했습니다.
   - A차량은 직진 중이었으며, B차량이 좌회전하면서 충돌이 발생했습니다. 일반적으로 비보호 좌회전 상황에서는 직진 차량의 우선권이 인정되므로, B차량의 과실이 일부 인정됩니다.
   - 문서의 사례에서는 B차량의 전방주시의무 태만이 인정되어 과실 30%가 부여되었습니다. 이 사례를 참고하여 B차량의 과실을 동일하게 30%로 판단합니다.

3. 관련 법률 조항:
   - 도로교통법 제27조 제1항 (교차로에서의 통행방법)
   - 도로교통법 제39조 (차로의 통행)

4. 참고 판례:
   - 대구지방법원 2010가단4991 (유사한 사고 상황에서 B차량의 과실 30%로 판단된 사례)


In [92]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

# run은 question 키를 기준으로 실행됨!
res2 = detail_chain.run("보행자우선도로가 뭔지 설명해줘")

print("✅ 기능② 답변:\n", res2)


✅ 기능② 답변:
 - 용어/조항 정의: 보행자우선도로는 보행자의 안전과 편의를 최우선으로 고려하여 설계된 도로를 의미합니다. 이 도로에서는 차량보다 보행자가 우선권을 가지며, 차량은 보행자의 안전을 위해 서행하거나 일시 정지해야 할 의무가 있습니다. 보행자우선도로는 주로 주거 지역이나 상업 지역 등 보행자가 많은 지역에 설치되며, 보행자의 안전을 강화하고 교통사고를 예방하기 위한 목적으로 운영됩니다.
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명이 제공되지 않았습니다.


In [165]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import re

# LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 기능 1번. 사고 상황 판단 프롬프트
prompt = PromptTemplate(
    template="""
너는 교통사고 과실 판단 전문가야.
아래 '사고 상황'을 분석하여 핵심 요소를 구조화하고, 반드시 문서 내에서 가장 유사한 사례(case)를 찾아 과실비율을 판단해줘.

---
사고 상황 원문:
{question}

➤ 사고 상황 요약 (다음 항목 기준):
- A차량 신호 및 진행 방식:
- B차량 신호 및 진행 방식:
- 충돌 방식 및 위치:
- 교차로/신호기 유무 등 도로 환경:

문서:
{context}

출력 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 판단 근거 요약
3. 적용 법률 (다음 형식으로 기재)
   - [법률명] 제[조]조 [항] (예: 도로교통법 제5조 제1항)
   - 여러 개면 각각 별도 줄로 나열
4. 참고 판례 (다음 형식으로 기재)
   - [법원명] [사건번호] (예: 대법원 2002다38767)
   - 여러 개면 각각 별도 줄로 나열

조건:
- 반드시 문서 내 유사 사례를 기반으로 판단해야 해.
- 사고 상황과 가장 유사한 사례를 먼저 정확히 찾고, 그 사례를 중심으로 판단 근거를 제시해.
- 직접적인 사례가 없으면, 가장 유사한 사례를 근거로 판단하고 그 차이를 설명해.
- 과실비율은 명확한 수치로 제시해.
- 법령, 판례는 반드시 문서에서 인용된 것을 정확한 형식으로 명시해.
- 무조건적인 추론이나 일반적인 상식은 사용하지 말고, 문서에 있는 정보로만 판단해.
- 판례명을 포함한 질문인 경우, 문서에서 사건번호와 법원명이 일치하는 문장을 찾아 정확히 인용해줘.
""",
    input_variables=["question", "context"]
)


# 기능 2번. 용어/법률 설명 프롬프트
detail_prompt = PromptTemplate(
    template="""
너는 교통사고와 법률 해설 전문 챗봇이야.

아래 문서 내용을 바탕으로 사용자 질문에 정확하고 간결하게 답변해줘.

질문:
{question}

--- 문서 내용 ---
{context}
------------------

📌 답변 작성 규칙:
1. 질문에 판례명(법원명 + 사건번호)이 포함된 경우:
   - 반드시 위 문서 내에서 **해당 사건번호가 포함된 문장**을 직접 인용해 설명해.
   - 문서에 해당 판례가 없으면 "**해당 판례는 문서에 포함되어 있지 않습니다**"라고 말해.

2. 질문이 용어나 법률 조항에 대한 경우:
   - 반드시 위 문서에 포함된 문장을 바탕으로 정의하거나 설명해.
   - 법률명, 조문 번호, 관련 판례명을 반드시 함께 명시해.

3. 문서에 없는 정보는 절대로 추측하거나 생성하지 마.

✍️ 출력 형식:
- 정의 또는 설명: [내용]
- 출처: [법률/조문 번호/판례명]

답변:
""",
    input_variables=["question", "context"]
)

# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 사고 상황 판단 체인
accident_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 설명용 체인
detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

def classify_query(user_input: str) -> str:
    accident_keywords = ["충돌", "사고", "과실", "A차량", "B차량", "신호위반", "부딪힘", "진입", "교차로"]
    explanation_keywords = ["무엇", "뭐야", "정의", "설명", "판례", "조항", "법원", "법", "도로교통법" ,"무슨 뜻", "의미", "용어", "이란"]

    # 기능② 설명 우선 처리
    if any(word in user_input for word in explanation_keywords):
        return "detail"
    if any(word in user_input for word in accident_keywords):
        return "accident"
    return "detail"


# # 판례명 추출 함수 (유지)
# def extract_court_name(text: str):
#     match = re.search(r"(대법원|서울고등법원|서울중앙지방법원|서울남부지방법원|[\w]+지방법원)", text)
#     return match.group() if match else None



# 통합 실행 함수
# 🔍 판례명 추출
def extract_case_id(text: str):
    match = re.search(r"(대법원|[\w]+고등법원|[\w]+지방법원)\s?\d{4}[가-힣]?\d+", text)
    return match.group() if match else None

# 🔍 법원명 추출
def extract_court_name(text: str):
    match = re.search(r"(대법원|[\w]+고등법원|[\w]+지방법원)", text)
    return match.group() if match else None

# 🧠 사고상황 vs 용어 설명 구분
def classify_query(user_input: str) -> str:
    accident_keywords = ["충돌", "사고", "과실", "A차량", "B차량", "신호위반", "부딪힘", "진입", "교차로"]
    explanation_keywords = ["무엇", "뭐야", "정의", "설명", "판례", "조항", "법원", "법", "도로교통법", "무슨 뜻", "의미", "용어", "이란"]

    if any(word in user_input for word in explanation_keywords):
        return "detail"
    if any(word in user_input for word in accident_keywords):
        return "accident"
    return "detail"

# ✨ context 최적화 함수
def build_context_from_matched_docs(matched_docs, key_phrase=None, limit=3):
    if key_phrase:
        primary_docs = [doc for doc in matched_docs if key_phrase in doc.page_content]
        other_docs = [doc for doc in matched_docs if key_phrase not in doc.page_content]
        ordered_docs = primary_docs + other_docs[:limit - len(primary_docs)]
    else:
        ordered_docs = matched_docs[:limit]

    # ✅ 각 문서 최대 800자 제한
    context = "\n\n".join(doc.page_content[:800] for doc in ordered_docs)
    return context

# ✅ 메인 실행 함수
def run_query(user_input: str):
    case_id = extract_case_id(user_input)
    court = extract_court_name(user_input)

    # ① 판례명 포함 질문
    if case_id:
        matched_docs = [doc for doc in all_docs if case_id in doc.page_content]

        if matched_docs:
            context = build_context_from_matched_docs(matched_docs, key_phrase=case_id)
            prompt_text = detail_prompt.format(question=user_input, context=context)
            result = model.invoke(prompt_text)
            print("📘 판례 직접 검색 결과:\n", result.content.strip())
            return
        else:
            print(f"❌ 해당 판례 '{case_id}'는 문서에 포함되어 있지 않습니다.")
            return

    # ② 법원명 기반 질문
    if court:
        matched_docs = [doc for doc in all_docs if doc.metadata.get("court") == court]

        if matched_docs:
            context = build_context_from_matched_docs(matched_docs, key_phrase=None)
            prompt_text = detail_prompt.format(question=user_input, context=context)
            result = model.invoke(prompt_text)
            print(f"📘 '{court}' 소속 판례 검색 결과:\n", result.content.strip())
            return
        else:
            print(f"❌ '{court}' 관련 문서가 포함되어 있지 않습니다.")
            return

    # ③ 일반 질문: 사고 상황 판단 or 용어 설명
    category = classify_query(user_input)

    if category == "accident":
        result = accident_chain.run(user_input)
        print("✅ 사고 상황 판단 결과:\n", result)
    else:
        result = detail_chain.run(user_input)
        print("📘 용어/법률 설명 결과:\n", result)





# # 통합 실행 함수
# def run_query(user_input: str):
#     category = classify_query(user_input)
#     if category == "accident":
#         result = accident_chain.run(user_input)
#         print("✅ 사고 상황 판단 결과:\n", result)
#     else:
#         result = detail_chain.run(user_input)
#         print("📘 용어/법률 설명 결과:\n", result)

### 테스트

In [166]:
run_query("A차량이 적색신호에 직진 중이고, B차량은 좌회전 신호 없이 녹색등에서 좌회전 중 충돌한 사고야. 과실비율은?")

✅ 사고 상황 판단 결과:
 사고 상황을 분석한 결과는 다음과 같습니다.

➤ 사고 상황 요약:
- A차량 신호 및 진행 방식: 적색신호에 직진 중
- B차량 신호 및 진행 방식: 좌회전 신호 없이 녹색등에서 좌회전 중
- 충돌 방식 및 위치: 교차로 내 충돌
- 교차로/신호기 유무 등 도로 환경: 교차로 내 신호기 존재

문서 내에서 가장 유사한 사례는 다음과 같습니다:
- "가 B차량이 녹색신호에 직진한 경우에는 A차량에 대하여 통행우선권이 있으나, A차량이 진행한 방향에는 신호기가 없으므로 신호위반의 과실을 묻기 어렵고, B차량은 이러한 교차로를 통과함에 있어 전방좌우를 철저히 살피고 안전운전을 해야 할 주의의무가 있으므로 양 차량의 기본 과실비율을 80:20으로 정한다."

이 사례와의 차이점은 B차량이 좌회전 중이었다는 점입니다. 따라서 B차량의 과실이 더 크다고 판단할 수 있습니다.

1. 과실비율: A차량 20% vs B차량 80%

2. 판단 근거 요약:
   - B차량이 녹색신호에서 좌회전한 점에서 과실이 크며, A차량은 적색신호에 직진한 점에서 과실이 있지만, 직진 차량으로서의 우선권이 일부 인정될 수 있음.

3. 적용 법률:
   - [도로교통법] 제5조 제1항

4. 참고 판례:
   - 해당 문서 내 직접적인 판례 정보 없음.


In [167]:
run_query("비보호 좌회전이 뭔지 설명해줘")

📘 용어/법률 설명 결과:
 - 정의 또는 설명: 비보호 좌회전은 좌회전하는 차량의 운전자가 반대방향에서 진행신호에 따라 직진하는 차량에 방해가 되지 않도록 유의하여 조심스럽게 좌회전을 해야 하는 상황을 의미합니다.
- 출처: [서울중앙지방법원 2017나65373]


In [168]:
run_query("대법원 92도2077 판결문 내용을 알려줘")

📘 '대법원' 소속 판례 검색 결과:
 - 정의 또는 설명: 차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대 차로 상에 정지하여 있는 차량의 뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상하여 그에 대한 주의의무를 다하여야 한다고는 할 수 없다.
- 출처: [대법원 92도2077]


In [169]:
run_query("서울남부지방법원에는 어떤 판례가 있어?")

📘 '서울남부지방법원' 소속 판례 검색 결과:
 - 정의 또는 설명: 서울남부지방법원 2018나64069: 신호등이 있는 ‘ㅏ’자형 교차로에서 전방 신호기 황색등화에 교차로를 가속 진입한 A차량과 반대편 차선 좌회전 금지구역에서 적색 신호에 좌회전을 한 B이륜차 간의 충돌로 B이륜차 운전자가 사망한 사고: B과실 55%(안전모 미착용 포함)
- 출처: [서울남부지방법원 2018나64069]

- 정의 또는 설명: 서울남부지방법원 2015가단227332: 주간에 신호등 및 비보호 좌회전 표지가 있는 삼거리(T자) 교차로에서 A차량이 선행 차량을 따라 서행으로 비보호 좌회전 하던 중 맞은편에서 직진신호에 따라 주행하여 오는 차량의 흐름을 잘 살펴 그 운행에 방해되지 않는 방법으로 안전하게 좌회전할 의무를 게을리 한 과실로, 교차로에 진입하기 전에 먼저 비보호 좌회전 하는 차량(제3차량)이 있었음에도 전혀 속도를 줄이지 아니한 채 만연히 신호에 따라 빠른 속도로 교차로에 진입하여 직진하던 B차량을 충격한 사고: B과실 10%
- 출처: [서울남부지방법원 2015가단227332]

- 정의 또는 설명: 서울남부지방법원 2018나11137: B 직진차량이 규정 속도 이내이고 양 차량 동시 교차로 진입하였고 A 비보호 좌회전 차량이 선행하던 직진 차량 뒤에서 급진입하여 좌회전한 경우 B차량이 A차량의 비보호 좌회전과의 충돌을 인지하고 브레이크를 밟아 제동이 시작되기까지의 공주거리에도 못 미치는 약 18m 전방에서 발견한 경우로, B차량 0%.(1심은 B차량 10%)
- 출처: [서울남부지방법원 2018나11137]


In [170]:
run_query("A차가 직진 중이고 B차가 좌회전하다 충돌했을 때 과실은?")

✅ 사고 상황 판단 결과:
 사고 상황을 분석하고 문서 내 유사 사례를 찾아 과실비율을 판단하겠습니다.

➤ 사고 상황 요약:
- A차량 신호 및 진행 방식: 직진 중
- B차량 신호 및 진행 방식: 좌회전 중
- 충돌 방식 및 위치: 충돌
- 교차로/신호기 유무 등 도로 환경: 불명

문서 내 유사 사례:
문서에서 제공된 사례 중 "서울중앙지방법원 2017나63155"는 A차량이 좌회전 중이고 B차량이 우회전 중인 경우로, A차량 20%의 과실이 인정되었습니다. 그러나, 이 사례는 A차량이 직진 중인 현재 사고 상황과 다소 차이가 있습니다. 따라서, 직진과 좌회전의 충돌 상황에 대한 직접적인 사례는 문서에 없으나, "대구지방법원 2010가단4991"의 경우, 신호등 없는 교차로에서 우회전과 직진 차량 간의 충돌 사례로, B차량에 30%의 과실이 인정되었습니다.

1. 과실비율: A차량 30% vs B차량 70%
2. 판단 근거 요약: 
   - A차량이 직진 중이고 B차량이 좌회전 중인 상황에서, 일반적으로 직진 차량에 우선권이 부여되며, 좌회전 차량의 주의 의무가 더 크다고 판단됩니다.
   - "대구지방법원 2010가단4991" 사례에서 우회전과 직진 차량 간의 충돌에서 직진 차량의 과실이 더 적게 인정된 점을 고려하여, 본 사고에서는 B차량의 과실이 더 크다고 판단하였습니다.
3. 적용 법률:
   - 도로교통법 제5조 제1항
4. 참고 판례:
   - 대구지방법원 2010가단4991

이와 같은 판단은 문서 내 유사 사례를 기반으로 한 것으로, 직접적인 사례가 없으므로 가장 유사한 사례의 과실비율을 참고하여 판단하였습니다.


In [171]:
run_query("서울남부지방법원 2018나64069 판결문 내용을 알려줘")

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 신호등이 있는 ‘ㅏ’자형 교차로에서 전방 신호기 황색등화에 교차로를 가속 진입한 A차량과 반대편 차선 좌회전 금지구역에서 적색 신호에 좌회전을 한 B이륜차 간의 충돌로 B이륜차 운전자가 사망한 사고: B과실 55%(안전모 미착용 포함)
- 출처: 서울남부지방법원 2018나64069


In [175]:
import random
json_precedent = load_json(file_paths["precedent"])
random_precedent = random.sample(json_precedent, 10)

for precedent in random_precedent:
    run_query(f"\n{precedent['court']} {precedent['case_id']} 판결문 내용을 알려줘")
    print()

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 자동차는 통행의 우선순위와는 관계없이 교통정리를 하고 있지 않고 좌우를 확인할 수 없거나 교통이 빈번한 교차로 등에서는 일시정지하여(도로교통법 제31조), 전방과 좌우를 잘 살펴 안전하게 교차로를 진입하고 통과하여야 할 주의의무가 있다고 할 것이지만, 교차로에 진입하면서 일단 전방 좌우를 살펴 안전하다는 판단하여 먼저 교차로에 진입한 이상 통행의 우선순위가 후순위인 차량의 통행법규위반 가능성도 예상하여 운전하여야 할 주의의무까지 있다고 할 수는 없다.
- 출처: 대법원 2013다64670

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 서울중앙지방법원 2017나76434 사건에 대한 판결문은 다음과 같습니다: "원고 차량이 편도 1차로 도로를 진행하다가 우측 노상 주차장에 주차하기 위하여 우회전하던 중 원고 차량의 후방에서 진행해오다 원고 차량과 같은 차로의 우측으로 원고차량을 앞지르려던 피고 이륜차와 충격한 사고에 대하여, 이 건 사고는 피고 이륜차가 앞지르기 방법을 위반하여 원고 차량의 도로 우측 틈으로 끼어들어 간 피고 이륜차의 주된 과실에 기인한 것이지만, 원고 차량도 우회전을 하려면 미리 도로의 우측 가장자리로 서행하면서 우회전을 하여야 하고, 우측 후사경을 통해 회전하고자 하는 우측 방면의 상태를 잘 살피고 방향지시등을 점등하여 주변 차량들로 하여금 진로변경을 예상할 수 있게 한 후 우회전을 할 주의의무가 있음에도 이러한 주의의무를 다하지 아니한 채 우회전을 한 과실이 있는 점을 고려함. 원고 차량의 과실 30%, 피고 이륜차의 과실 70%."
- 출처: [서울중앙지방법원 2017나76434]

📘 '서울고등법원' 소속 판례 검색 결과:
 - 정의 또는 설명: **해당 판례는 문서에 포함되어 있지 않습니다**
- 출처: [질문에 포함된 판례명]

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 신호기에 의하여 교통정리가 행하여지고 있는 교차로의 전방에 노면표지로서 유턴을 허용하는 안전표지가 설치